In [1]:
import requests 
import base64

In [4]:
CLIENT_ID = 'bdf2419d541d4a7cb08af790c66d484f'
CLIENT_SECRET = 'd67c3900605e43d7b21d4fb78cf07fdb'

client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())    

In [5]:
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url,data=data,headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print('access token obtained successfully.')
else:
    print('error obtaining access token.')
    exit()

access token obtained successfully.


In [6]:
pip install spotipy

     -------------------------------------- 241.1/241.1 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [18]:
def get_trending_playlist_data(playlist_id,access_token):
    sp = spotipy.Spotify(auth=access_token)
    playlist_tracks = sp.playlist_tracks(playlist_id, fields = 'items(track(id,name,artists,album(id,name)))')
    
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']
        
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not Available' else None
        
        try:
            album_info = sp.album(album_id) if album_id != 'Not Available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None
            
        try:
            track_info = sp.track(track_id) if track_id != 'Not Available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None
            
        track_data = {
            'Track Name':track_name,
            'Artists':artists,
            'Album Name':album_name,
            'Track ID':track_id,
            'Popularity':popularity,
            'Release Date':release_date,
            'Duration(ms)':audio_features['duration_ms'] if audio_features else None,
            'Explicit':track_info.get('explicit',None),
            'External URLs':track_info.get('external_urls',{}).get('spotify',None),
            'Danceability':audio_features['danceability'] if audio_features else None,
            'Energy':audio_features['energy'] if audio_features else None,
            'Key':audio_features['key'] if audio_features else None,
            'Loudness':audio_features['loudness'] if audio_features else None,
            'Mode':audio_features['mode'] if audio_features else None,
            'Speechiness':audio_features['speechiness'] if audio_features else None,
            'Acousticness':audio_features['acousticness'] if audio_features else None,
            'Instrumentalness':audio_features['instrumentalness'] if audio_features else None,
            'Liveness':audio_features['liveness'] if audio_features else None,
            'Valence':audio_features['valence'] if audio_features else None,
            'Tempo':audio_features['tempo'] if audio_features else None,
        }    
        
        music_data.append(track_data)
    df = pd.DataFrame(music_data)
    return df

In [20]:
playlist_id = '2237sMNMlXS4wWLgdQ1UuV?si=YD8Vy5zxSsiRCtNzd14i-w'

music = get_trending_playlist_data(playlist_id,access_token)

print(music)

SpotifyException: http status: 400, code:-1 - Unsupported URL / URI., reason: None